In [1]:
import os
import random
import torch
import matplotlib.pyplot as plt
import polars as pl
import numpy as np
from collections import defaultdict

from kinpfn.priors import Batch
from kinpfn.model import KINPFN


from sklearn.mixture import BayesianGaussianMixture, GaussianMixture
from sklearn.neighbors import KernelDensity

def set_seed(seed=123):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [2]:
def get_dataset_size(val_set_dir):
    dataset_size = 0
    for subdir, _, files in os.walk(val_set_dir):
        for file in files:
            if file.endswith(".csv"):
                dataset_size += 1
    return dataset_size


def get_batch_testing_folding_times(val_set_dir, seq_len=100, num_features=1, **kwargs):

    dataset_size = get_dataset_size(val_set_dir)

    x = torch.zeros(seq_len, dataset_size, num_features)
    y = torch.zeros(seq_len, dataset_size)

    batch_index = 0
    length_order = []
    file_names_in_order = []
    for subdir, _, files in os.walk(val_set_dir):
        for file in files:
            if file.endswith(".csv"):
                path = os.path.join(subdir, file)
                data = pl.read_csv(
                    path,
                    has_header=False,
                    columns=[2, 4],
                    n_rows=seq_len,
                )

                folding_times = data["column_3"].to_numpy()
                sequence = data["column_5"][0]
                length_order.append(len(sequence))
                file_names_in_order.append(file)
                sorted_folding_times = np.sort(folding_times)

                # Filter out points where x > 10^15 and x < 10^-6
                valid_indices = (sorted_folding_times <= 10**15) & (
                    sorted_folding_times >= 10**-6
                )
                sorted_folding_times = sorted_folding_times[valid_indices]

                # Adjust the sequence length by sampling
                current_seq_len = len(sorted_folding_times)
                if current_seq_len <= 0:
                    continue

                if current_seq_len < seq_len:
                    # Repeat the sorted_folding_times and cdf to match the sequence length (Oversampling)
                    repeat_factor = seq_len // current_seq_len + 1
                    sorted_folding_times = np.tile(sorted_folding_times, repeat_factor)[
                        :seq_len
                    ]
                else:
                    sorted_folding_times = sorted_folding_times[:seq_len]

                x[:, batch_index, 0] = torch.tensor(np.zeros(seq_len))
                y[:, batch_index] = torch.tensor(sorted_folding_times)
                batch_index += 1

    y = torch.log10(y)
    return Batch(x=x, y=y, target_y=y), length_order, file_names_in_order

In [ ]:
model_path = "../../../../models/final_kinpfn_model_1400_1000_1000_86_50_2.5588748050825984e-05_256_4_512_8_0.0_0.0.pt"

kinpfn = KINPFN(
    model_path=model_path,
)
trained_model = kinpfn.model

if trained_model is not None:
    print("Load trained model!")
else:
    print("No trained model found!")
    exit()

In [4]:
def group_lengths_and_losses(mean_losses, std_losses, bin_ranges):
    grouped_mean_losses = defaultdict(list)
    grouped_std_losses = defaultdict(list)

    for length, loss in mean_losses.items():
        for bin_key in bin_ranges:
            if bin_key[0] <= length <= bin_key[1]:
                grouped_mean_losses[bin_key].append(loss)
                grouped_std_losses[bin_key].append(std_losses[length])
                break

    grouped_mean_losses = {
        key: torch.mean(torch.tensor(val)) for key, val in grouped_mean_losses.items()
    }
    grouped_std_losses = {
        key: torch.sqrt(torch.sum(torch.tensor(val) ** 2) / len(val))
        for key, val in grouped_std_losses.items()
    }

    return grouped_mean_losses, grouped_std_losses


def plot_losses(
    grouped_nll_losses,
    grouped_std_nll_losses,
    grouped_mae_losses,
    grouped_std_mae_losses,
):
    fontsize = 25
    labelsize = 30

    lengths = sorted(grouped_nll_losses.keys(), key=lambda x: x[0])
    nll_values = [grouped_nll_losses[length] for length in lengths]
    nll_stds = [grouped_std_nll_losses[length] for length in lengths]
    mae_values = [grouped_mae_losses[length] for length in lengths]
    mae_stds = [grouped_std_mae_losses[length] for length in lengths]

    fig, ax1 = plt.subplots(figsize=(27, 11))
    fig.set_dpi(300)

    bar_width = 0.35
    index = np.arange(len(lengths))

    ax1.bar(
        index,
        nll_values,
        bar_width,
        yerr=nll_stds,
        capsize=5,
        label="Mean NLL incl. Std.",
        color="darkred",
        zorder=-10,
    )
    ax1.set_xlabel("RNA Sequence Length Ranges", fontsize=labelsize)
    ax1.set_ylabel("Mean NLL", color="darkred", fontsize=labelsize)
    ax1.tick_params(axis="y", labelcolor="darkred", labelsize=fontsize)
    ax1.tick_params(axis="x", labelsize=fontsize)
    ax2 = ax1.twinx()
    ax2.bar(
        index + bar_width,
        mae_values,
        bar_width,
        yerr=mae_stds,
        capsize=5,
        label="MAE incl. Std.",
        color="steelblue",
        zorder=-10,
    )
    ax2.set_ylabel("MAE", color="steelblue", fontsize=labelsize)
    ax2.tick_params(axis="y", labelcolor="steelblue", labelsize=fontsize)
    legend1 = ax1.legend(loc="upper left", fontsize=fontsize)
    legend1.set_zorder(10)
    ax1.add_artist(legend1)
    legend2 = ax2.legend(loc="upper right", fontsize=fontsize)
    legend2.set_zorder(10)
    ax2.add_artist(legend2)

    plt.xticks(
        index + bar_width / 2, [f"{length[0]}-{length[1]}" for length in lengths]
    )
    fig.tight_layout()
    plt.show()
    plt.close("all")


def model_testing_losses(trained_model):
    seed = 75342
    print(f"Seed: {seed}")
    set_seed(seed=seed)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    seq_len = 1000
    training_points = [25]

    val_set_dir = "../../../../kinpfn_testing_set"

    batch, length_order, file_names_in_order = get_batch_testing_folding_times(
        val_set_dir=val_set_dir, seq_len=seq_len
    )
    dataset_size = get_dataset_size(val_set_dir)
    print(f"Dataset Size: {dataset_size}")

    x = batch.x
    y_folding_times = batch.y
    target_y_folding_times = batch.target_y

    indices = list(range(dataset_size))

    for training_point in training_points:
        print(f"Training Point: {training_point}")

        nll_losses_per_length = defaultdict(list)
        mae_losses_per_length = defaultdict(list)

        for i in indices:
            batch_index = i
            length = length_order[batch_index]

            train_indices = torch.randperm(seq_len)[:training_point]
            test_indices = ~train_indices

            # Create the training and test data
            train_x = x[train_indices, batch_index]
            train_y_folding_times = y_folding_times[train_indices, batch_index]

            test_x = x[:, batch_index]
            test_y_folding_times = y_folding_times[:, batch_index]

            train_x = train_x.to(device)
            train_y_folding_times = train_y_folding_times.to(device)
            test_x = test_x.to(device)
            test_y_folding_times = test_y_folding_times.to(device)

            with torch.no_grad():
                # we add our batch dimension, as our transformer always expects that
                logits = trained_model(
                    train_x[:, None], train_y_folding_times[:, None], test_x[:, None]
                )

            ### CALCULATE GROUND TRUTH CDF WHICH WE WANT TO PREDICT
            ground_truth_sorted_folding_times, _ = torch.sort(test_y_folding_times)
            ground_truth_cdf = torch.arange(
                1, len(ground_truth_sorted_folding_times) + 1
            ) / len(ground_truth_sorted_folding_times)

            test_y_folding_times_sorted, _ = torch.sort(test_y_folding_times)

            ### CDF FUNCTION
            pred_cdf_original = (
                trained_model.criterion.cdf(logits, test_y_folding_times_sorted)
            )[0][0]

            # MAE
            single_absolute_error = np.abs(pred_cdf_original - ground_truth_cdf)
            mae = single_absolute_error.mean()
            mae_losses_per_length[length].append(mae)

            # NLL
            nll_loss = trained_model.criterion.forward(
                logits=logits, y=test_y_folding_times_sorted
            )
            mean_nll_loss = nll_loss.mean()
            nll_losses_per_length[length].append(mean_nll_loss)

        mean_nll_per_length = {
            length: torch.mean(torch.stack(nlls))
            for length, nlls in nll_losses_per_length.items()
        }
        std_nll_per_length = {
            length: torch.std(torch.stack(nlls)) if len(nlls) > 1 else torch.tensor(0.0)
            for length, nlls in nll_losses_per_length.items()
        }

        mean_mae_per_length = {
            length: torch.mean(torch.stack(maes))
            for length, maes in mae_losses_per_length.items()
        }
        std_mae_per_length = {
            length: torch.std(torch.stack(maes)) if len(maes) > 1 else torch.tensor(0.0)
            for length, maes in mae_losses_per_length.items()
        }
        bin_ranges = [
            (15, 19),
            (20, 29),
            (30, 39),
            (40, 49),
            (50, 59),
            (60, 69),
            (70, 79),
            (80, 89),
            (90, 99),
            (100, 109),
            (110, 119),
            (120, 129),
            (130, 139),
            (140, 147),
        ]
        grouped_nll_losses, grouped_std_nll_losses = group_lengths_and_losses(
            mean_nll_per_length, std_nll_per_length, bin_ranges=bin_ranges
        )
        grouped_mae_losses, grouped_std_mae_losses = group_lengths_and_losses(
            mean_mae_per_length, std_mae_per_length, bin_ranges=bin_ranges
        )

        plot_losses(
            grouped_nll_losses,
            grouped_std_nll_losses,
            grouped_mae_losses,
            grouped_std_mae_losses,
        )

In [5]:
# Plot the losses of the model through various RNA sequence length ranges on the testing set
#model_testing_losses(trained_model)

In [6]:
def model_testing_losses_kde():
    seed = 75342
    print(f"Seed: {seed}")
    set_seed(seed=seed)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    seq_len = 1000
    training_points = [25]

    val_set_dir = "../../../../kinpfn_testing_set"

    batch, length_order, file_names_in_order = get_batch_testing_folding_times(
        val_set_dir=val_set_dir, seq_len=seq_len
    )
    dataset_size = get_dataset_size(val_set_dir)
    print(f"Dataset Size: {dataset_size}")

    x = batch.x
    y_folding_times = batch.y
    target_y_folding_times = batch.target_y

    indices = list(range(dataset_size))

    for training_point in training_points:
        print(f"Training Point: {training_point}")

        nll_losses_per_length = defaultdict(list)
        mae_losses_per_length = defaultdict(list)

        for i in indices:
            batch_index = i
            length = length_order[batch_index]

            train_indices = torch.randperm(seq_len)[:training_point]
            test_indices = ~train_indices

            # Create the training and test data
            train_x = x[train_indices, batch_index]
            train_y_folding_times = y_folding_times[train_indices, batch_index]

            test_x = x[:, batch_index]
            test_y_folding_times = y_folding_times[:, batch_index]

            train_x = train_x.to(device)
            train_y_folding_times = train_y_folding_times.to(device)
            test_x = test_x.to(device)
            test_y_folding_times = test_y_folding_times.to(device)

            with torch.no_grad():
                # we add our batch dimension, as our transformer always expects that
                logits = trained_model(
                    train_x[:, None], train_y_folding_times[:, None], test_x[:, None]
                )

            ### CALCULATE GROUND TRUTH CDF WHICH WE WANT TO PREDICT
            ground_truth_sorted_folding_times, _ = torch.sort(test_y_folding_times)
            ground_truth_cdf = torch.arange(
                1, len(ground_truth_sorted_folding_times) + 1
            ) / len(ground_truth_sorted_folding_times)

            test_y_folding_times_sorted, _ = torch.sort(test_y_folding_times)

            ### CDF FUNCTION
            # pred_cdf_original = (
            #     trained_model.criterion.cdf(logits, test_y_folding_times_sorted)
            # )[0][0]

            # # MAE
            # single_absolute_error = np.abs(pred_cdf_original - ground_truth_cdf)
            # mae = single_absolute_error.mean()
            # mae_losses_per_length[length].append(mae)

            # # NLL
            # nll_loss = trained_model.criterion.forward(
            #     logits=logits, y=test_y_folding_times_sorted
            # )
            # mean_nll_loss = nll_loss.mean()
            # nll_losses_per_length[length].append(mean_nll_loss)


            # KDE MAE and NLL
            kde_train_y_folding_times = train_y_folding_times.reshape(-1, 1).cpu().numpy()
            kde = KernelDensity(kernel='gaussian', bandwidth=0.3520031472796679).fit(kde_train_y_folding_times)
            kde_test_y_folding_times_sorted = test_y_folding_times_sorted.reshape(-1, 1)
            kde_pdf = np.exp(kde.score_samples(kde_test_y_folding_times_sorted))
            kde_cdf = np.cumsum(kde_pdf) / np.sum(kde_pdf)
            kde_mae = np.abs(kde_cdf - ground_truth_cdf.cpu().numpy()).mean()
            # to tensor
            kde_mae = torch.tensor(kde_mae)
            mae_losses_per_length[length].append(kde_mae)


            kde_nll = -np.mean(kde.score_samples(kde_test_y_folding_times_sorted))
            kde_nll = torch.tensor(kde_nll)
            nll_losses_per_length[length].append(kde_nll)



        mean_nll_per_length = {
            length: torch.mean(torch.stack(nlls))
            for length, nlls in nll_losses_per_length.items()
        }
        std_nll_per_length = {
            length: torch.std(torch.stack(nlls)) if len(nlls) > 1 else torch.tensor(0.0)
            for length, nlls in nll_losses_per_length.items()
        }

        mean_mae_per_length = {
            length: torch.mean(torch.stack(maes))
            for length, maes in mae_losses_per_length.items()
        }
        std_mae_per_length = {
            length: torch.std(torch.stack(maes)) if len(maes) > 1 else torch.tensor(0.0)
            for length, maes in mae_losses_per_length.items()
        }
        bin_ranges = [
            (15, 19),
            (20, 29),
            (30, 39),
            (40, 49),
            (50, 59),
            (60, 69),
            (70, 79),
            (80, 89),
            (90, 99),
            (100, 109),
            (110, 119),
            (120, 129),
            (130, 139),
            (140, 147),
        ]
        grouped_nll_losses, grouped_std_nll_losses = group_lengths_and_losses(
            mean_nll_per_length, std_nll_per_length, bin_ranges=bin_ranges
        )
        grouped_mae_losses, grouped_std_mae_losses = group_lengths_and_losses(
            mean_mae_per_length, std_mae_per_length, bin_ranges=bin_ranges
        )

        plot_losses(
            grouped_nll_losses,
            grouped_std_nll_losses,
            grouped_mae_losses,
            grouped_std_mae_losses,
        )

In [7]:
#model_testing_losses_kde()

In [8]:
def group_lengths_and_losses_kde_kinpfn(mean_losses, std_losses, bin_ranges):
    grouped_mean_losses = defaultdict(list)
    grouped_std_losses = defaultdict(list)

    for length, loss in mean_losses.items():
        for bin_key in bin_ranges:
            if bin_key[0] <= length <= bin_key[1]:
                grouped_mean_losses[bin_key].append(loss)
                grouped_std_losses[bin_key].append(std_losses[length])
                break

    grouped_mean_losses = {
        key: torch.mean(torch.tensor(val)) for key, val in grouped_mean_losses.items()
    }
    grouped_std_losses = {
        key: torch.sqrt(torch.sum(torch.tensor(val) ** 2) / len(val))
        for key, val in grouped_std_losses.items()
    }

    return grouped_mean_losses, grouped_std_losses


def plot_losses_kde_kinpfn(
    kinpfn_grouped_losses,
    kinpfn_grouped_std_losses,
    kde_grouped_losses,
    kde_grouped_std_losses,
    lossType="NLL",
):
    fontsize = 25
    labelsize = 30

    lengths = sorted(kinpfn_grouped_losses.keys(), key=lambda x: x[0])
    kinpfn_values = [kinpfn_grouped_losses[length] for length in lengths]
    kinpfn_stds = [kinpfn_grouped_std_losses[length] for length in lengths]
    kde_values = [kde_grouped_losses[length] for length in lengths]
    kde_stds = [kde_grouped_std_losses[length] for length in lengths]

    fig, ax1 = plt.subplots(figsize=(27, 11))
    fig.set_dpi(300)

    bar_width = 0.35
    index = np.arange(len(lengths))

    ax1.bar(
        index,
        kinpfn_values,
        bar_width,
        yerr=kinpfn_stds,
        capsize=5,
        label=f"KinPFN {lossType} incl. Std.",
        color="darkred",
        zorder=-10,
    )
    ax1.set_xlabel("RNA Sequence Length Ranges", fontsize=labelsize)
    ax1.set_ylabel('KinPFN' + lossType, color="darkred", fontsize=labelsize)
    ax1.tick_params(axis="y", labelcolor="darkred", labelsize=fontsize)
    ax1.tick_params(axis="x", labelsize=fontsize)
    ax2 = ax1.twinx()
    ax2.bar(
        index + bar_width,
        kde_values,
        bar_width,
        yerr=kde_stds,
        capsize=5,
        label=f"KDE {lossType} incl. Std.",
        color="steelblue",
        zorder=-10,
    )
    ax2.set_ylabel('KDE' + lossType, color="steelblue", fontsize=labelsize)
    ax2.tick_params(axis="y", labelcolor="steelblue", labelsize=fontsize)
    legend1 = ax1.legend(loc="upper left", fontsize=fontsize)
    legend1.set_zorder(10)
    ax1.add_artist(legend1)
    legend2 = ax2.legend(loc="upper right", fontsize=fontsize)
    legend2.set_zorder(10)
    ax2.add_artist(legend2)

    plt.xticks(
        index + bar_width / 2, [f"{length[0]}-{length[1]}" for length in lengths]
    )
    fig.tight_layout()
    plt.show()
    plt.close("all")


def model_testing_losses_kde_kinpfn(trained_model):
    seed = 75342
    print(f"Seed: {seed}")
    set_seed(seed=seed)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    seq_len = 1000
    training_points = [25]

    val_set_dir = "../../../../kinpfn_testing_set"

    batch, length_order, file_names_in_order = get_batch_testing_folding_times(
        val_set_dir=val_set_dir, seq_len=seq_len
    )
    dataset_size = get_dataset_size(val_set_dir)
    print(f"Dataset Size: {dataset_size}")

    x = batch.x
    y_folding_times = batch.y
    target_y_folding_times = batch.target_y

    indices = list(range(dataset_size))

    for training_point in training_points:
        print(f"Training Point: {training_point}")

        nll_losses_per_length = defaultdict(list)
        mae_losses_per_length = defaultdict(list)

        kde_nll_losses_per_length = defaultdict(list)
        kde_mae_losses_per_length = defaultdict(list)

        for i in indices:
            batch_index = i
            length = length_order[batch_index]

            train_indices = torch.randperm(seq_len)[:training_point]
            test_indices = ~train_indices

            # Create the training and test data
            train_x = x[train_indices, batch_index]
            train_y_folding_times = y_folding_times[train_indices, batch_index]

            test_x = x[:, batch_index]
            test_y_folding_times = y_folding_times[:, batch_index]

            train_x = train_x.to(device)
            train_y_folding_times = train_y_folding_times.to(device)
            test_x = test_x.to(device)
            test_y_folding_times = test_y_folding_times.to(device)

            with torch.no_grad():
                # we add our batch dimension, as our transformer always expects that
                logits = trained_model(
                    train_x[:, None], train_y_folding_times[:, None], test_x[:, None]
                )

            ### CALCULATE GROUND TRUTH CDF WHICH WE WANT TO PREDICT
            ground_truth_sorted_folding_times, _ = torch.sort(test_y_folding_times)
            ground_truth_cdf = torch.arange(
                1, len(ground_truth_sorted_folding_times) + 1
            ) / len(ground_truth_sorted_folding_times)

            test_y_folding_times_sorted, _ = torch.sort(test_y_folding_times)

            ### CDF FUNCTION
            pred_cdf_original = (
                trained_model.criterion.cdf(logits, test_y_folding_times_sorted)
            )[0][0]

            # MAE
            single_absolute_error = np.abs(pred_cdf_original - ground_truth_cdf)
            mae = single_absolute_error.mean()
            mae_losses_per_length[length].append(mae)

            # NLL
            nll_loss = trained_model.criterion.forward(
                logits=logits, y=test_y_folding_times_sorted
            )
            mean_nll_loss = nll_loss.mean()
            nll_losses_per_length[length].append(mean_nll_loss)



            # KDE MAE and NLL
            kde_train_y_folding_times = train_y_folding_times.reshape(-1, 1).cpu().numpy()
            kde = KernelDensity(kernel='gaussian', bandwidth=0.3520031472796679).fit(kde_train_y_folding_times)
            kde_test_y_folding_times_sorted = test_y_folding_times_sorted.reshape(-1, 1)
            kde_pdf = np.exp(kde.score_samples(kde_test_y_folding_times_sorted))
            kde_cdf = np.cumsum(kde_pdf) / np.sum(kde_pdf)
            kde_mae = np.abs(kde_cdf - ground_truth_cdf.cpu().numpy()).mean()
            # to tensor
            kde_mae = torch.tensor(kde_mae)
            kde_mae_losses_per_length[length].append(kde_mae)


            kde_nll = -np.mean(kde.score_samples(kde_test_y_folding_times_sorted))
            kde_nll = torch.tensor(kde_nll)
            kde_nll_losses_per_length[length].append(kde_nll)

        mean_nll_per_length = {
            length: torch.mean(torch.stack(nlls))
            for length, nlls in nll_losses_per_length.items()
        }
        std_nll_per_length = {
            length: torch.std(torch.stack(nlls)) if len(nlls) > 1 else torch.tensor(0.0)
            for length, nlls in nll_losses_per_length.items()
        }

        kde_mean_nll_per_length = {
            length: torch.mean(torch.stack(nlls))
            for length, nlls in kde_nll_losses_per_length.items()
        }
        kde_std_nll_per_length = {
            length: torch.std(torch.stack(nlls)) if len(nlls) > 1 else torch.tensor(0.0)
            for length, nlls in kde_nll_losses_per_length.items()
        }

        mean_mae_per_length = {
            length: torch.mean(torch.stack(maes))
            for length, maes in mae_losses_per_length.items()
        }
        std_mae_per_length = {
            length: torch.std(torch.stack(maes)) if len(maes) > 1 else torch.tensor(0.0)
            for length, maes in mae_losses_per_length.items()
        }

        kde_mean_mae_per_length = {
            length: torch.mean(torch.stack(maes))
            for length, maes in kde_mae_losses_per_length.items()
        }
        kde_std_mae_per_length = {
            length: torch.std(torch.stack(maes)) if len(maes) > 1 else torch.tensor(0.0)
            for length, maes in kde_mae_losses_per_length.items()
        }

        bin_ranges = [
            (15, 19),
            (20, 29),
            (30, 39),
            (40, 49),
            (50, 59),
            (60, 69),
            (70, 79),
            (80, 89),
            (90, 99),
            (100, 109),
            (110, 119),
            (120, 129),
            (130, 139),
            (140, 147),
        ]
        grouped_nll_losses, grouped_std_nll_losses = group_lengths_and_losses_kde_kinpfn(
            mean_nll_per_length, std_nll_per_length, bin_ranges=bin_ranges
        )
        grouped_mae_losses, grouped_std_mae_losses = group_lengths_and_losses_kde_kinpfn(
            mean_mae_per_length, std_mae_per_length, bin_ranges=bin_ranges
        )

        kde_grouped_nll_losses, kde_grouped_std_nll_losses = group_lengths_and_losses_kde_kinpfn(
            kde_mean_nll_per_length, kde_std_nll_per_length, bin_ranges=bin_ranges
        )
        kde_grouped_mae_losses, kde_grouped_std_mae_losses = group_lengths_and_losses_kde_kinpfn(
            kde_mean_mae_per_length, kde_std_mae_per_length, bin_ranges=bin_ranges
        )

        plot_losses_kde_kinpfn(
            grouped_nll_losses,
            grouped_std_nll_losses,
            kde_grouped_nll_losses,
            kde_grouped_std_nll_losses,
        )

        plot_losses_kde_kinpfn(
            grouped_mae_losses,
            grouped_std_mae_losses,
            kde_grouped_mae_losses,
            kde_grouped_std_mae_losses,
            lossType="MAE",
        )

In [ ]:
model_testing_losses_kde_kinpfn(trained_model)